In [ ]:
import sys
print("Python Version:", sys.version, '\n')

# Deep and Shallow Copy

The way Python treats variables is a bit frustrating at times. We're going to fully examine how it works today. The first thing to remember is that the variable doesn't contain the value. What it contains is the memory location of the value. So we'll need to play some games to make sure we get what we want when assigning things to variables.

Let's start by looking at an example with just an integer.

In [ ]:
a = 5
b = a
a = a + 7

print(b)

Well, that behaved about as we expected. Now let's try it with a list.

In [ ]:
a = [1,2,3,4]
b = a
print(b)

In [ ]:
a.append(7)
print(b)

Whoa. Changing `a`, actually changed `b`. Is that what we expected. What if we do this one?

In [ ]:
a = [5,6,7,8]
print(b)

In [ ]:
a = [1,2,3,4]
b = a
a[0] = 5
print(b)

<img src="deep_copy_demo/deep_copy_demo.001.jpeg"  style="max-width:50%; float:right;">

Python variables work by assigning *names* to values.

* Immutable objects (int, float, tuple, etc) live in one spot in memory, and multiple names can point to them.
* Mutable ojects (lists, dictionaries, dataframes) are the same, but you can change them, so changing what one name "holds" affects what other "names" access.

So changing one might affect the other. Let's look at a cartoon of the memory.


What if we want to make a new list that looks like `a`? Will it do the same thing as above?

In [ ]:
a = [1,2,3,4]
b = list(a)
a[0] = 4
print(a)
print(b)

<img src="deep_copy_demo/deep_copy_demo.002.jpeg"  style="max-width:50%; float:right;">

Wait, why did that work?



When we called `list(a)`, we called a constructor which asked Python to make a whole new list of the values `a` contained. That's great. 

#### So if that's the case, something like this should work.

In [ ]:
a = [1,2,[3,4],5]
b = list(a)
a[2][0] = 1
print(b)

<img src="deep_copy_demo/deep_copy_demo.003.jpeg" style="max-width:50%; float:right;">

Well. Uhh. That's weird then. What's happened? Let's go back to our cartoon of the memory.

The copy we made made a copy of each element, including the list, however it made an exact copy of the list. Meaning, the list it made just points to the same elements that the list in `a` does. The nested structure didn't get a duplicate made, just the top-level list. So our nested data (the integers inside the second list) is still just at one specific location in memory. So our copies share an address that they point to.

This is known as a "shallow copy." From the documentation on different types of copying:

>The difference between shallow and deep copying is only relevant for compound objects (objects that contain other objects, like lists or class instances):
>
>- A shallow copy constructs a new compound object and then (to the extent possible) inserts references into it to the objects found in the original.
>- A deep copy constructs a new compound object and then, recursively, inserts copies into it of the objects found in the original.

In [ ]:
a = [1,2,[3,6],4,5]
b = list(a)
a[0] = 5
print(b) # Note that b doesn't change it's 0th spot

In [ ]:
a[2][0] = 9
print(b) # Note that b does change it's internal list spot 0!

In [ ]:
from copy import deepcopy
a = [1,2,[3,6],4,5]
b = deepcopy(a)

In [ ]:
a[2][0] = 9
print(b) # Note that it doesn't change once we deep copy!

### Application to pandas and series

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame([[1,2,3],[4,5,6]],columns=['a','b','c'])
df

In [ ]:
# Remember that shallow copy happens for compound objects
# And pandas dataframes are compound objects made up of series! 

# What happens if we make a copy of a df and change a column? 
new_df = df
new_df['a'] = [8,9]
df

Hmm, looks like column 'a' of our original pandas dataframe changed! 

In [ ]:
# To do a deep copy with a pandas dataframe, use copy() 

newer_df = df.copy()
newer_df['a'] = ['y','z']
newer_df

In [ ]:
# And what does the original df look like? Does it have [y,z] or [8,9] in the firt column? 

df

In [ ]:
df[df['a'] < 9]['b'] = 42

Why did that give us an error? That's because df[conditional] returned to us only a copy of the dataframe. 

In [ ]:
df

If we actually wanted to change the values, we would use loc, which does not access a copy but accesses the actual object. 

In [ ]:
df.loc[df['a']<9, 'b'] = 42
df

<img src="deep_copy_demo/deep_copy_demo.004.jpeg"  style="max-width:50%; float:right;">
Deep copy takes up more memory than a shallow copy, as it recreates everything recursively. However, that can be safer and necessary when working with highly nested data. If in doubt, deep copy. Unless the data's huge.

In cartoon format, our deep copy looks like:


### Additional Resources:
- [Python Documentation](https://docs.python.org/2/library/copy.html)
- [Video of copy types](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=video&cd=2&cad=rja&uact=8&ved=0ahUKEwiH78iJkOLYAhVE7GMKHb3ODmsQtwIILTAB&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3Di7ePjqf6Y-Y&usg=AOvVaw3pH-AYImBuStUn91Ap_gz4)